# Πρώτη επαφή με το Google Earth Engine

1. Οδηγίες ενεργοποίησης και πρόσβασης στο GEE: https://developers.google.com/earth-engine/guides/access
2. Authentication and Initialization: https://developers.google.com/earth-engine/guides/auth

In [1]:
import ee # Google Earth Engine Python Client
import geemap # frontend του ee Google Earth Engine


# gee authentication
ee.Authenticate()
ee.Initialize(project='your-project-id')


In [2]:
#  VIIRS Nighttime Day/Night Band Composites Version 1
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG

# Δημιουργία χάρτη με geemap (ipyleaflet) 
Map = geemap.Map(center=[39.07, 21.82], zoom=6)

# Ανάκτηση εικόνων VIIRS DNB με φίλτρο για τον Μάιο του 2017
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
.filter(ee.Filter.date('2017-05-01', '2017-05-31'))

# Επιλογή του καναλιού avg_rad για οπτικοποίηση
nighttime = dataset.select('avg_rad')
# Οπτικοποίηση με κατάλληλα min/max για το κανάλι avg_rad
nighttimeVis = {"min": 0.0, "max": 60.0}

# Κεντράρισμα του χάρτη και προσθήκη του επιπέδου με τις παραμέτρους οπτικοποίησης
Map.setCenter(-77.1056, 38.8904, 8)
Map.addLayer(nighttime, nighttimeVis, 'Nighttime')
Map

Map(center=[38.8904, -77.1056], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

In [3]:
# VIIRS VNP46A2 — Greece (2020)


# Γεωμετρία Ελλάδας από το dataset: FAO GAUL (level0)

# FeatureCollection
greece_fc = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(
    ee.Filter.eq('ADM0_NAME', 'Greece')
)
# Ανάκτηση γεωμετρίας
greece = greece_fc.geometry()

# ανάκτηση εικόνων VIIRS VNP46A2 με φίλτρο για το 2020 στα όρια της Ελλάδας
viirs = ee.ImageCollection('NASA/VIIRS/002/VNP46A2') \
    .filterDate('2020-01-01', '2021-01-01') \
    .filterBounds(greece)

print('Αριθμός εικόνων VIIRS το 2020 (για την Ελλάδα):', viirs.size().getInfo())

# Επιλέγουμε την πρώτη εικόνα για να δούμε τα διαθέσιμα bands

first = ee.Image(viirs.first())
bands = first.bandNames().getInfo()
print('Διαθέσιμα κανάλια:', bands)



# Δημιουργία median composite και αποκοπή στο περίγραμμα της Ελλάδας
median = viirs.median().select('Gap_Filled_DNB_BRDF_Corrected_NTL').clip(greece)

# Παραμετροποίηση οπτικοποίησης
vis = {
    'min': 0,
    'max': 100,
    'palette': ['black', 'purple', 'blue', 'cyan', 'green', 'yellow', 'white']
}

# Δημιουργία χάρτη με geemap και προσθήκη των επιπέδων
m = geemap.Map(center=[39.07, 21.82], zoom=6)
m.addLayer(median, vis, 'VIIRS VNP46A2 median 2020 (Greece)')
m.addLayer(greece_fc.style(**{'color': 'red', 'fillColor': '00000000'}), {}, 'Greece boundary')

# Προβολή του χάρτη
m

Αριθμός εικόνων VIIRS το 2020 (για την Ελλάδα): 366
Διαθέσιμα κανάλια: ['DNB_BRDF_Corrected_NTL', 'DNB_Lunar_Irradiance', 'Gap_Filled_DNB_BRDF_Corrected_NTL', 'Latest_High_Quality_Retrieval', 'Mandatory_Quality_Flag', 'QF_Cloud_Mask', 'Snow_Flag']


Map(center=[39.07, 21.82], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…